# Volatility Surface Fitting

This notebook demonstrates fitting SVI volatility surfaces to options data.

**Contents:**
1. Load and Prepare Data
2. SVI Model Overview
3. Interactive Surface Fitting
4. Fit Quality Analysis
5. 3D Surface Visualization

In [ ]:
# Standard imports
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import date, timedelta

# Volsurf imports
from volsurf.database.connection import get_connection
from volsurf.database.schema import init_schema
from volsurf.models.svi import SVIModel, SVIParams
from volsurf.fitting.pipeline import SurfaceFittingPipeline
from volsurf.analytics import get_vol_at_moneyness

# Initialize
init_schema()
conn = get_connection()

SYMBOL = 'SPY'

## 1. Load Options Data

In [ ]:
# Get latest date with data
latest_date = conn.execute(
    "SELECT MAX(quote_date) FROM raw_options_chains WHERE symbol = ?",
    [SYMBOL]
).fetchone()[0]

print(f"Fitting surfaces for {SYMBOL} on {latest_date}")

# Load liquid options
options_df = conn.execute("""
    SELECT 
        expiration_date,
        strike,
        option_type,
        bid, ask, mid,
        implied_volatility,
        open_interest,
        volume,
        underlying_price
    FROM raw_options_chains
    WHERE symbol = ? AND quote_date = ? AND is_liquid = TRUE
    ORDER BY expiration_date, strike
""", [SYMBOL, latest_date]).fetchdf()

underlying_price = options_df['underlying_price'].iloc[0]
print(f"Underlying: ${underlying_price:.2f}")
print(f"Liquid options: {len(options_df):,}")
print(f"Expirations: {options_df['expiration_date'].nunique()}")

## 2. SVI Model Overview

The SVI (Stochastic Volatility Inspired) model parameterizes total implied variance as:

$$w(k) = a + b \cdot \left( \rho (k - m) + \sqrt{(k - m)^2 + \sigma^2} \right)$$

Where:
- $k = \ln(K/F)$ is log-moneyness
- $a$ = vertical shift (ATM variance level)
- $b$ = slope of the wings
- $\rho$ = skew parameter ($-1 < \rho < 1$)
- $m$ = horizontal shift
- $\sigma$ = smoothness parameter

In [ ]:
# Visualize SVI with different parameters
k = np.linspace(-0.3, 0.3, 100)
tte = 30/365  # 30 days

# Base parameters
base_params = {'a': 0.04, 'b': 0.1, 'rho': -0.3, 'm': 0.0, 'sigma': 0.1}

fig = go.Figure()

# Base case
vol_base = [get_vol_at_moneyness(base_params['a'], base_params['b'], base_params['rho'], 
                                  base_params['m'], base_params['sigma'], tte, ki) for ki in k]
fig.add_trace(go.Scatter(x=k, y=[v*100 for v in vol_base], name='Base (rho=-0.3)'))

# Higher skew
vol_skew = [get_vol_at_moneyness(base_params['a'], base_params['b'], -0.6, 
                                  base_params['m'], base_params['sigma'], tte, ki) for ki in k]
fig.add_trace(go.Scatter(x=k, y=[v*100 for v in vol_skew], name='Higher skew (rho=-0.6)'))

# Symmetric smile
vol_sym = [get_vol_at_moneyness(base_params['a'], base_params['b'], 0.0, 
                                 base_params['m'], base_params['sigma'], tte, ki) for ki in k]
fig.add_trace(go.Scatter(x=k, y=[v*100 for v in vol_sym], name='Symmetric (rho=0)'))

fig.update_layout(
    title='SVI Model: Effect of Skew Parameter (rho)',
    xaxis_title='Log-Moneyness (k)',
    yaxis_title='Implied Volatility (%)',
    hovermode='x unified'
)
fig.show()

## 3. Fit Surface for Single Expiration

In [ ]:
# Select an expiration to fit
expirations = sorted(options_df['expiration_date'].unique())
print("Available expirations:")
for i, exp in enumerate(expirations[:10]):
    dte = (exp - latest_date).days
    count = len(options_df[options_df['expiration_date'] == exp])
    print(f"  [{i}] {exp} ({dte} DTE, {count} options)")

In [ ]:
# Choose expiration (change index as needed)
selected_exp = expirations[2]  # ~30 DTE typically
exp_df = options_df[options_df['expiration_date'] == selected_exp].copy()

dte = (selected_exp - latest_date).days
tte = dte / 365

print(f"Selected: {selected_exp} ({dte} DTE)")
print(f"Options: {len(exp_df)}")

In [ ]:
# Prepare data for fitting
# Use OTM options (calls for K > S, puts for K < S)
exp_df['is_otm'] = (
    ((exp_df['option_type'] == 'CALL') & (exp_df['strike'] >= underlying_price)) |
    ((exp_df['option_type'] == 'PUT') & (exp_df['strike'] <= underlying_price))
)

otm_df = exp_df[exp_df['is_otm']].copy()

# Calculate log-moneyness
otm_df['log_moneyness'] = np.log(otm_df['strike'] / underlying_price)

# Filter valid IVs
otm_df = otm_df[otm_df['implied_volatility'].notna() & (otm_df['implied_volatility'] > 0)]

print(f"OTM options for fitting: {len(otm_df)}")

In [ ]:
# Fit SVI model
svi_model = SVIModel()

strikes = otm_df['strike'].values
ivs = otm_df['implied_volatility'].values

params = svi_model.fit(
    strikes=strikes,
    implied_vols=ivs,
    forward_price=underlying_price,  # Approximating forward with spot
    time_to_expiry=tte
)

print("\nFitted SVI Parameters:")
print(f"  a = {params.a:.6f}")
print(f"  b = {params.b:.6f}")
print(f"  rho = {params.rho:.6f}")
print(f"  m = {params.m:.6f}")
print(f"  sigma = {params.sigma:.6f}")
print(f"\nFit Quality:")
print(f"  RMSE = {params.rmse:.4%}")

In [ ]:
# Plot fitted vs market
k_range = np.linspace(otm_df['log_moneyness'].min(), otm_df['log_moneyness'].max(), 100)
fitted_vols = [get_vol_at_moneyness(params.a, params.b, params.rho, params.m, params.sigma, tte, ki) 
               for ki in k_range]

fig = go.Figure()

# Market data
fig.add_trace(go.Scatter(
    x=otm_df['log_moneyness'],
    y=otm_df['implied_volatility'] * 100,
    mode='markers',
    name='Market IV',
    marker=dict(size=8)
))

# Fitted curve
fig.add_trace(go.Scatter(
    x=k_range,
    y=[v*100 for v in fitted_vols],
    mode='lines',
    name=f'SVI Fit (RMSE={params.rmse:.3%})',
    line=dict(color='red', width=2)
))

fig.add_vline(x=0, line_dash='dash', line_color='gray', annotation_text='ATM')

fig.update_layout(
    title=f'{SYMBOL} Volatility Smile - {selected_exp} ({dte} DTE)',
    xaxis_title='Log-Moneyness',
    yaxis_title='Implied Volatility (%)',
    hovermode='x unified'
)

fig.show()

## 4. Fit All Expirations

In [ ]:
# Use the fitting pipeline
pipeline = SurfaceFittingPipeline()

# Fit all expirations
results = pipeline.fit_all_expirations(SYMBOL, latest_date)

print(f"Fitted {len(results)} expirations")

In [ ]:
# Display results
results_df = pd.DataFrame([{
    'Expiration': r.expiration_date,
    'DTE': int(r.tte_years * 365),
    'ATM Vol': f"{r.atm_vol:.2%}" if r.atm_vol else 'N/A',
    'Skew (rho)': f"{r.svi_rho:.4f}",
    'RMSE': f"{r.rmse:.4%}",
    'Points': r.num_points
} for r in results])

results_df

## 5. 3D Surface Visualization

In [ ]:
# Build 3D surface
moneyness_grid = np.linspace(-0.25, 0.25, 50)
tte_grid = np.array([r.tte_years for r in results])

iv_surface = np.zeros((len(results), len(moneyness_grid)))

for i, r in enumerate(results):
    for j, k in enumerate(moneyness_grid):
        iv = get_vol_at_moneyness(r.svi_a, r.svi_b, r.svi_rho, r.svi_m, r.svi_sigma, r.tte_years, k)
        iv_surface[i, j] = iv if not np.isnan(iv) else 0

# Create 3D plot
fig = go.Figure(data=[go.Surface(
    x=moneyness_grid,
    y=tte_grid * 365,
    z=iv_surface * 100,
    colorscale='Viridis',
    colorbar=dict(title='IV (%)')
)])

fig.update_layout(
    title=f'{SYMBOL} Fitted Volatility Surface ({latest_date})',
    scene=dict(
        xaxis_title='Log-Moneyness',
        yaxis_title='Days to Expiry',
        zaxis_title='Implied Vol (%)',
        camera=dict(eye=dict(x=1.5, y=-1.5, z=1.2))
    ),
    width=900,
    height=700
)

fig.show()

## Summary

Key observations from surface fitting:

1. **Fit Quality**: [Add observations about RMSE]
2. **Skew**: [Add observations about rho across expirations]
3. **Term Structure**: [Add observations about ATM vol term structure]